In [1]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize Mediapipe face detection and drawing utilities
mp_face_mesh = mp.solutions.face_mesh

# Load mask images (transparent PNGs)
mask_images = {
    'ironman': cv2.imread('ironman.png', cv2.IMREAD_UNCHANGED),
    'eyemask': cv2.imread('eyemask.png', cv2.IMREAD_UNCHANGED),
    'batman': cv2.imread('batman.png', cv2.IMREAD_UNCHANGED),
    'hulk': cv2.imread('hulk.png', cv2.IMREAD_UNCHANGED),
    'mask': cv2.imread('mask.png', cv2.IMREAD_UNCHANGED),
    'spiderman': cv2.imread('spiderman.png', cv2.IMREAD_UNCHANGED),
    'coolers': cv2.imread('coolers.png', cv2.IMREAD_UNCHANGED),
}

# Check if the masks are loaded properly
for name, img in mask_images.items():
    if img is None:
        raise ValueError(f"{name.capitalize()} mask image not found. Please check the file path.")

# Default selected mask
current_mask = mask_images['ironman']

# Function to rotate an image
# Function to rotate an image around a specific center, with adjustments for different mask types
def rotate_image(image, angle, mask_type):
    (h, w) = image.shape[:2]
    
    if mask_type == 'batman':
        # Batman-specific rotation center
        center = (w // 1.7, h // 1.3)
    else:
        # Default center for other masks
        center = (w // 2, h // 2)

    # Rotate the image around the calculated center
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    return rotated


# Function to overlay one image (mask) on top of another (face)
def overlay_mask(face_image, mask_image, location, size, angle=0, mask_type=None):
    # Resize mask to match face size, adjusting width and height scaling
    if mask_type == 'ironman':
        mask_resized = cv2.resize(mask_image, (int(size[0] * 1.7), int(size[1] * 1.7)))
        x_offset = 0.18
        y_offset = 0.45
    elif mask_type == 'hulk':
        mask_resized = cv2.resize(mask_image, (int(size[0] * 1.5), int(size[1] * 1.5)))
        x_offset = 0.18
        y_offset = 0.45
    elif mask_type == 'spiderman':
        mask_resized = cv2.resize(mask_image, (int(size[0] * 1.5), int(size[1] * 1.5)))
        x_offset = 0.15
        y_offset = 0.45
    elif mask_type == 'coolers':
        mask_resized = cv2.resize(mask_image, (int(size[0] * 1.15), int(size[1] * 0.40)))
        x_offset = 0.05
        y_offset = -0.17
    elif mask_type == 'eyemask':
        mask_resized = cv2.resize(mask_image, (int(size[0] * 1.3), int(size[1] * 1.0)))
        x_offset = 0.10
        y_offset = 0.25
    elif mask_type == 'batman':
        mask_resized = cv2.resize(mask_image, (int(size[0] * 1.5), int(size[1] * 1.5)))
        x_offset = 0.18
        y_offset = 0.59
    elif mask_type == 'mask':  # General mask to handle all 5 types
        mask_resized = cv2.resize(mask_image, (int(size[0] * 1.35), int(size[1] * 0.9)))
        x_offset = 0.13
        y_offset = 0.24
    else:
        raise ValueError("Invalid mask type")

    # Rotate the mask based on the face's angle and mask type
    mask_rotated = rotate_image(mask_resized, angle, mask_type)

    # Separate the color and alpha channels of the mask image
    mask_rgb = mask_rotated[:, :, :3]
    mask_alpha = mask_rotated[:, :, 3]
    x, y = location

    # Center the mask horizontally on the face
    x -= int(x_offset * mask_rotated.shape[1])  # Adjust horizontal position (move left)
    
    # Shift the mask upwards to align better with the face
    y -= int(y_offset * size[1])  # Move the mask upwards relative to the face size

    # Ensure the coordinates are within the frame boundaries
    h, w, _ = mask_rgb.shape
    h_frame, w_frame, _ = face_image.shape

    if y + h > h_frame:
        h = h_frame - y
    if x + w > w_frame:
        w = w_frame - x

    # Resize the mask again if it exceeds the frame dimensions
    mask_rgb = mask_rgb[:h, :w]
    mask_alpha = mask_alpha[:h, :w]

    # Get the region of interest (ROI) where the mask will be placed
    roi = face_image[y:y + h, x:x + w]
    for c in range(0, 3):  # For each color channel
        roi[:, :, c] = (mask_rgb[:, :, c] * (mask_alpha / 255.0) + roi[:, :, c] * (1.0 - mask_alpha / 255.0))
    face_image[y:y + h, x:x + w] = roi

# Function to calculate the angle between the eyes
def calculate_face_angle(landmarks, image_width, image_height):
    left_eye = landmarks[33]  # Left eye landmark (from Mediapipe Face Mesh)
    right_eye = landmarks[263]  # Right eye landmark (from Mediapipe Face Mesh)

    left_eye_x = int(left_eye.x * image_width)
    left_eye_y = int(left_eye.y * image_height)
    right_eye_x = int(right_eye.x * image_width)
    right_eye_y = int(right_eye.y * image_height)

    # Calculate the angle in degrees between the eyes
    dx = right_eye_x - left_eye_x
    dy = right_eye_y - left_eye_y
    angle = np.degrees(np.arctan2(dy, dx))

    # Invert angle to match mirrored webcam feed
    return -angle  # Flip the angle to correct the rotation direction

# Function to handle mouse click events
def on_mouse_click(event, x, y, flags, param):
    global current_mask
    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if the click is within the region of the mask icons
        icon_height = 100
        icon_width = 100
        icon_y_start = frame.shape[0] - icon_height  # Position icons below the video feed
        icon_names = list(mask_images.keys())
        
        for i, name in enumerate(icon_names):
            if icon_y_start <= y <= icon_y_start + icon_height:
                if i * icon_width <= x <= (i + 1) * icon_width:
                    current_mask = mask_images[name]
                    print(f"{name.capitalize()} mask selected!")

# Initialize webcam
cap = cv2.VideoCapture(0)

# Set mouse callback for the window
cv2.namedWindow('Snapchat Mask Selector')
cv2.setMouseCallback('Snapchat Mask Selector', on_mouse_click)

with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Flip the frame horizontally (like a mirror) to match the real-world movement
        frame = cv2.flip(frame, 1)
        # Convert the BGR image to RGB for Mediapipe processing
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image to detect faces and facial landmarks
        results = face_mesh.process(image_rgb)

        # If a face is detected, process the first face
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                ih, iw, _ = frame.shape

                # Calculate the bounding box around the face
                x_min = int(min([lm.x for lm in face_landmarks.landmark]) * iw)
                y_min = int(min([lm.y for lm in face_landmarks.landmark]) * ih)
                x_max = int(max([lm.x for lm in face_landmarks.landmark]) * iw)
                y_max = int(max([lm.y for lm in face_landmarks.landmark]) * ih)

                w = x_max - x_min
                h = y_max - y_min

                # Calculate the face angle
                angle = calculate_face_angle(face_landmarks.landmark, iw, ih)

                # Overlay selected mask onto the face
                mask_size = (w, h)  # Match mask size to face bounding box
                mask_type = [name for name, img in mask_images.items() if np.array_equal(img, current_mask)][0]
                overlay_mask(frame, current_mask, (x_min, y_min), mask_size, angle, mask_type)

        # Draw mask icons below the video feed
        icon_height = 91
        icon_width = 91
        icon_y_start = frame.shape[0] - icon_height  # Position icons below the video feed
        num_masks = len(mask_images)  # Get the number of masks
        
        # Resize icons and draw them
        for i, (name, img) in enumerate(mask_images.items()):
            icon = cv2.resize(img, (icon_width, icon_height))
            icon_rgb = icon[:, :, :3]
            frame[icon_y_start:icon_y_start + icon_height, i * icon_width:(i + 1) * icon_width] = icon_rgb

        # Show the final image with the mask overlay
        cv2.imshow('Snapchat Mask Selector', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Eyemask mask selected!
Batman mask selected!
Hulk mask selected!
Hulk mask selected!
Hulk mask selected!
Mask mask selected!
Spiderman mask selected!
Mask mask selected!
Spiderman mask selected!
Coolers mask selected!
